# Production Line Optimization Exercise

In this exercise, you will solve a production line optimization problem using linear programming. You will:
1. Define decision variables
2. Create constraints
3. Build the objective function
4. Solve and analyze the results

## Problem Statement

TechCraft builds custom gaming PCs for North American customers. They need to optimize their production across multiple facilities to maximize profit while meeting demand constraints.

### Production Facilities:
- Primary facilities (dedicated lines, must be used):
  * Taiwan facility (T1): 1,800 units/month
  * Mexico facility (M1): 6,000 units/month
  * California facility (C): 1,500 units/month
- Optional facilities (can be repurposed):
  * Taiwan second line (T2): 2,200 units/month
  * Mexico second line (M2): 2,000 units/month

### Fixed Costs for Repurposing:
- Taiwan second line (T2): $21,100
- Mexico second line (M2): $41,700

### Expected Demand:
- Las Vegas (WH1): 6,000 units
- Denver (WH2): 5,200 units

### Profit per Unit ($):
| Production Line | Las Vegas (WH1) | Denver (WH2) | Capacity (units/month) |
|----------------|-----------------|---------------|----------------------|
| Taiwan (T1)    | 46              | 49           | 1,800               |
| Mexico (M1)    | 30              | 35           | 6,000               |
| California (C) | 82              | 83           | 1,500               |
| Taiwan 2 (T2)  | 44              | 52           | 2,200               |
| Mexico 2 (M2)  | 86              | 81           | 2,000               |

## Setup

First, let's import the necessary packages and define our test functions.

In [ ]:
import pulp
import numpy as np
import pandas as pd
from optimization_tests import check_variables, check_constraints, check_objective

## Step 1: Define the Model and Variables

Create the PuLP model and define the decision variables. You need to create:

1. A PuLP maximization problem
2. Decision variables for units produced at each facility for each warehouse (hint: use `LpVariable.dicts`)
3. Binary variables for optional production lines

### Graded Cell
Complete the code below:

In [ ]:
# Define production lines and warehouses
production_lines = ['T1', 'M1', 'C', 'T2', 'M2']
warehouses = ['WH1', 'WH2']

# YOUR CODE HERE
# 1. Create the model
model = None

# 2. Create decision variables for production
vars = None

# 3. Create binary variables for optional lines
use_line = None

In [ ]:
# Test your variable definitions
check_variables(model, vars, use_line)

## Step 2: Define Constraints

Now add the following constraints to your model:
1. Demand constraints for each warehouse
2. Capacity constraints for each production line
3. Optional line usage constraint (only one can be used)
4. Link optional lines to their usage

### Graded Cell
Complete the code below:

In [ ]:
# Define capacities
capacities = {
    'T1': 1800,
    'M1': 6000,
    'C': 1500,
    'T2': 2200,
    'M2': 2000
}

# YOUR CODE HERE
# 1. Add demand constraints

# 2. Add capacity constraints

# 3. Add optional line constraint

# 4. Link optional lines to usage


In [ ]:
# Test your constraints
check_constraints(model, vars, use_line, capacities)

## Step 3: Define the Objective Function

Create the objective function that maximizes total profit:
1. Calculate revenue from all production lines
2. Subtract fixed costs for optional lines used

### Graded Cell
Complete the code below:

In [ ]:
# Define profits and fixed costs
profits = {
    ('T1', 'WH1'): 46, ('T1', 'WH2'): 49,
    ('M1', 'WH1'): 30, ('M1', 'WH2'): 35,
    ('C', 'WH1'): 82, ('C', 'WH2'): 83,
    ('T2', 'WH1'): 44, ('T2', 'WH2'): 52,
    ('M2', 'WH1'): 86, ('M2', 'WH2'): 81
}

fixed_costs = {
    'T2': 21100,
    'M2': 41700
}

# YOUR CODE HERE
# Add the objective function to the model


In [ ]:
# Test your objective function
check_objective(model, vars, use_line, profits, fixed_costs)

## Step 4: Solve and Analyze Results

Now that you've built the complete model, let's solve it and analyze the results!

In [ ]:
# Solve the model
model.solve()

# Create results dataframe
results = []
for i in production_lines:
    total_production = sum(vars[i,j].value() for j in warehouses)
    if total_production > 0:
        for j in warehouses:
            if vars[i,j].value() > 0:
                results.append({
                    'Production Line': i,
                    'Warehouse': j,
                    'Units': int(vars[i,j].value()),
                    'Capacity': capacities[i],
                    'Capacity Utilization': f"{(total_production/capacities[i])*100:.1f}%",
                    'Profit per Unit': profits[i,j],
                    'Total Profit': int(vars[i,j].value() * profits[i,j])
                })

df_results = pd.DataFrame(results)
print("Production Plan:")
print(df_results)

# Calculate total profit including fixed costs
total_profit = pulp.value(model.objective)
fixed_costs_used = sum(fixed_costs[line] * use_line[line].value() for line in ['T2', 'M2'])

print(f"\nProfit from Production: ${total_profit + fixed_costs_used:,.2f}")
print(f"Fixed Costs: ${fixed_costs_used:,.2f}")
print(f"Net Profit: ${total_profit:,.2f}")

# Show which optional line was used
print("\nOptional Lines Used:")
for line in ['T2', 'M2']:
    if use_line[line].value() == 1:
        print(f"- {line} is active (Fixed Cost: ${fixed_costs[line]:,})")